In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

print(chat.predict("How do you make italian pasta?"))


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [5.98s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the dough into desired shapes such as fettuccine, sp

In [11]:
print(chat.predict("How do you make italian pasta?"))


To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it becomes smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, divide the dough into smaller portions and roll out each portion into a thin sheet using a pasta machine or a rolling pin.
7. Cut the pasta sheet into desired shapes such as fettuccine, spaghetti, or ravioli.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your homemade Italian pasta!
